# Diluted Unitary Fitting Calibration

In [3]:
import sys
sys.path.insert(0, '../../../src_tf/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import pickle as pkl
import tensorflow as tf

from qiskit.quantum_info import DensityMatrix, random_unitary
from qiskit.quantum_info import Operator
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm
from math import ceil

from kraus_channels import KrausMap, DilutedKrausMap
from loss_functions import ProbabilityMSE, ProbabilityRValue, channel_fidelity_loss, SpectrumDistance
from optimization import ModelSPAM, ModelQuantumMap, Logger, model_saver
from quantum_channel import DilutedChannel, ChoiMapStatic
from quantum_tools import apply_unitary, measurement, resample, state_fidelity, generate_unitary
from experimental import counts_to_probs, generate_pauliInput_circuits, generate_pauli_circuits
from spam import SPAM, InitialState, POVMwQR as POVM, IdealPOVM, IdealInit, povm_fidelity, CorruptionMatrix
from synthetic_data import generate_spam_data, generate_map_data
from utils import saver, loader, train_val_split
from quantum_circuits import pqc_basic
from spectrum import channel_spectrum, mean_spacing, spectrum_distance


#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=4)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [1]:
def spectrum_distance_benchmark(d, c, rank, reps):
    distance_list = []

    for _ in tqdm(range(reps)):
        map1 = DilutedKrausMap(U=generate_unitary(d), c=c, kraus_part=KrausMap(d, rank=rank, trainable=False))
        spectrum1 = channel_spectrum(map1)

        map2 = DilutedKrausMap(U=generate_unitary(d), c=c, kraus_part=KrausMap(d, rank=rank, trainable=False))
        spectrum2 = channel_spectrum(map2)

        sigma = mean_spacing(spectrum1)

        distance = spectrum_distance(spectrum1, spectrum2, sigma)
        distance_list.append(distance)
    distance_mean = np.mean(distance_list)
    distance_std = np.std(distance_list)
        
    return distance_mean, distance_std


## Four Qubits